In [32]:
import os
import pandas as pd
from transformers import pipeline
import numpy as np

In [2]:
classifier = pipeline("text-classification", 
                      model="j-hartmann/emotion-english-distilroberta-base", 
                      return_all_scores=True)

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/home/ucloud/.local/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [3]:
classifier("amazing job, well done")

[[{'label': 'anger', 'score': 0.004060383420437574},
  {'label': 'disgust', 'score': 0.0007226790767163038},
  {'label': 'fear', 'score': 0.0023012515157461166},
  {'label': 'joy', 'score': 0.8008537888526917},
  {'label': 'neutral', 'score': 0.022170746698975563},
  {'label': 'sadness', 'score': 0.004300907254219055},
  {'label': 'surprise', 'score': 0.1655903160572052}]]

In [4]:
data_path = os.path.join("..","data")
data_name = "Game_of_Thrones_Script.csv"
data_path = os.path.join(data_path,data_name)
data = pd.read_csv(data_path)


#take lines, make predictions, append predictions to og df
sentences = data["Sentence"]

In [9]:
for line in lines:
    classifier(line)



23911

In [57]:
test = classifier(sentences[1])

In [16]:
pd.DataFrame(test)

,0,1,2,3,4,5,6
0,"{'label': 'anger', 'score': 0.0947076603770256}","{'label': 'disgust', 'score': 0.8400837182998657}","{'label': 'fear', 'score': 0.005907748825848103}","{'label': 'joy', 'score': 0.001032969681546092}","{'label': 'neutral', 'score': 0.04298973456025...","{'label': 'sadness', 'score': 0.00452361674979...","{'label': 'surprise', 'score': 0.0107545163482..."


In [58]:
#the model output is a list of dictionaries, so values are extracted and reformatted
label_list = []
score_list = []
#get labels and scores
for emotion_output in test[0]:
    label = emotion_output["label"]
    score = round(emotion_output["score"],2)
    #add them to their respective lists
    label_list.append(label)
    score_list.append(score)
#make score list a numpy array, reshape from column to row
row = np.array(score_list).reshape(1,7)

In [98]:
def extract_emotion_scores(line, n_outputs = 7):
    """A function which takes the output from an emotion classifier model, and formats its
    scores to a numpy array for easier handling. 
    line: a single output from an emotional classifier model 

    n_outputs: The function works on the basis, that 7 scores are in the output.

    """
    #the first layer of the output is removed
    line = line[0]
    #the model output is a list of dictionaries, so values are extracted and reformatted
    score_list = []
    #get scores
    for emotion_output in line:
        score = round(emotion_output["score"],2)
        #add them to their respective lists
        label_list.append(label)
        score_list.append(score)
    #make score list a numpy array, reshape from column to row
    row = np.array(score_list).reshape(1,n_outputs)
    return row

In [64]:
fos1 = extract_emotion_scores(classifier(sentences[0]))
fos2 = extract_emotion_scores(classifier(sentences[1]))

In [66]:
foslist = [fos1,fos2]

In [77]:
lines = sentences[0:30]

In [79]:

def getlabels(line):
    #the first layer of the output is removed
    line = line[0]
    #the model output is a list of dictionaries, so values are extracted and reformatted
    label_list = []
    #get scores
    for emotion_output in line:
        label = emotion_output["label"]
        #add them to their respective lists
        label_list.append(label)
    return label_list

In [104]:
len(data["Sentence"])

23911

66.41944444444444

In [89]:

#get labels
labels = getlabels(classifier(lines[0]))
#get scores
#counter to help organize arrays, everything is appended to 1st
counter = 0
#each line is classified, then scores are extracted as numpy arrays of dimension (1,7)
#then they are appended so the it becomes (2,7) then (3,7) and so on..
for line in lines:
    if counter == 0:
        prediction_final =  extract_emotion_scores(classifier(line))
    else:
        prediction_temporary =  extract_emotion_scores(classifier(line))

        prediction_final  = np.append(prediction_final,prediction_temporary, axis=0)
    
    counter += 1
    

In [99]:
def multiline_extract_emotion_scores(classifier,lines, n_outputs = 7):
    #counter to help organize arrays, everything is appended to 1st
    counter = 0
    #each line is classified, then scores are extracted as numpy arrays of dimension (1,7)
    #then they are appended so the it becomes (2,7) then (3,7) and so on..
    for line in lines:
        if counter == 0:
            prediction_final =  extract_emotion_scores(classifier(line),n_outputs)
        else:
            prediction_temporary =  extract_emotion_scores(classifier(line),n_outputs)

            prediction_final  = np.append(prediction_final,prediction_temporary, axis=0)
        
        counter += 1
    return prediction_final

In [100]:
fos = multiline_extract_emotion_scores(classifier,sentences[:10])

In [101]:
fos

array([[0.09, 0.84, 0.01, 0.  , 0.04, 0.  , 0.01],
       [0.08, 0.61, 0.02, 0.  , 0.13, 0.01, 0.16],
       [0.05, 0.02, 0.03, 0.  , 0.17, 0.02, 0.71],
       [0.02, 0.04, 0.01, 0.01, 0.91, 0.01, 0.01],
       [0.14, 0.1 , 0.42, 0.  , 0.28, 0.05, 0.01],
       [0.01, 0.01, 0.96, 0.  , 0.02, 0.  , 0.01],
       [0.16, 0.03, 0.11, 0.02, 0.62, 0.05, 0.01],
       [0.07, 0.04, 0.01, 0.01, 0.09, 0.01, 0.77],
       [0.56, 0.3 , 0.04, 0.  , 0.08, 0.01, 0.  ],
       [0.64, 0.04, 0.22, 0.  , 0.08, 0.02, 0.  ]])

In [93]:
test_scores = pd.DataFrame(prediction_final, columns = labels)
test_df = data[:30]

In [96]:
test_df.join(test_scores)

,Release Date,Season,Episode,Episode Title,Name,Sentence,anger,disgust,fear,joy,neutral,sadness,surprise
0,2011-04-17,Season 1,Episode 1,Winter is Coming,waymar royce,What do you expect? They're savages. One lot s...,0.09,0.84,0.01,0.00,0.04,0.00,0.01
1,2011-04-17,Season 1,Episode 1,Winter is Coming,will,I've never seen wildlings do a thing like this...,0.08,0.61,0.02,0.00,0.13,0.01,0.16
2,2011-04-17,Season 1,Episode 1,Winter is Coming,waymar royce,How close did you get?,0.05,0.02,0.03,0.00,0.17,0.02,0.71
3,2011-04-17,Season 1,Episode 1,Winter is Coming,will,Close as any man would.,0.02,0.04,0.01,0.01,0.91,0.01,0.01
4,2011-04-17,Season 1,Episode 1,Winter is Coming,gared,We should head back to the wall.,0.14,0.10,0.42,0.00,0.28,0.05,0.01
5,2011-04-17,Season 1,Episode 1,Winter is Coming,royce,Do the dead frighten you?,0.01,0.01,0.96,0.00,0.02,0.00,0.01
6,2011-04-17,Season 1,Episode 1,Winter is Coming,gared,Our orders were to track the wildlings. We tra...,0.16,0.03,0.11,0.02,0.62,0.05,0.01
7,2011-04-17,Season 1,Episode 1,Winter is Coming,royce,You don't think he'll ask us how they died? Ge...,0.07,0.04,0.01,0.01,0.09,0.01,0.77
8,2011-04-17,Season 1,Episode 1,Winter is Coming,will,Whatever did it to them could do it to us. The...,0.56,0.30,0.04,0.00,0.08,0.01,0.00
9,2011-04-17,Season 1,Episode 1,Winter is Coming,royce,It's a good thing we're not children. You want...,0.64,0.04,0.22,0.00,0.08,0.02,0.00


In [76]:
pd.DataFrame(fostot, columns = label_list)

,anger,disgust,fear,joy,neutral,sadness,surprise
0,0.09,0.84,0.01,0.0,0.04,0.00,0.01
1,0.08,0.61,0.02,0.0,0.13,0.01,0.16
2,0.09,0.84,0.01,0.0,0.04,0.00,0.01
